In [60]:
import pandas as pd
import geopandas as gpd
import json as js
import matplotlib.pyplot as plt
import re
import numpy as np
import folium
from folium import IFrame
from shapely.geometry import Polygon
from bs4 import BeautifulSoup
from pykml import parser

In [61]:
fuel_df = pd.read_csv('alt_fuel_stations (Jul 29 2021).csv')
fuel_df.head()

/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_22752/2859254457.py:1: DtypeWarning: Columns (3,6,10,13,14,15,16,20,29,31,33,34,35,36,43,45,46,47,49,52,53,54,56,57,58,59,60,61,62,63) have mixed types. Specify dtype option on import or set low_memory=False.
  fuel_df = pd.read_csv('alt_fuel_stations (Jul 29 2021).csv')


,Fuel Type Cotion Directions,City,State,ZIP,Plus4,Station Phone,Status Code,Expected Date,Groups With Access Code,Access Days Time,...,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Accessde,Station Name,Street Address,Intersec
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,NaN
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Clean Energy - Texas Department of Transportation,7721A Washington St,"I-10, Washington Ave exit, 1.5 blocks to the s...",Houston,TX,77007,NaN,866-809-4869,E,...,NaN,NaN,NaN,NaN,Q,3000 3600,MD,NaN,NaN,NaN


In [62]:
#Cleaning the Data
#Identify the columns to keep from original df
columns_to_keep = ['City', 'ID', 'Updated At', 'Fuel Type Cotion Directions', 'Plus4', 'State', 'Station Phone']  

# Select only the specified columns
cleaned_df = fuel_df[columns_to_keep]
    
# Print the DataFrame with selected columns
cleaned_df.sample(15)
type(cleaned_df['City'])
type(cleaned_df['Updated At'])

# I need to remake new columns to make the DF easier to understand
cleaned_df['lat'] = cleaned_df['ID']
cleaned_df['lon'] = cleaned_df['Updated At']
cleaned_df['Address'] = cleaned_df['State']
cleaned_df['Station_Name'] = cleaned_df['City']
cleaned_df['Fuel Type'] = cleaned_df['Fuel Type Cotion Directions']
cleaned_df['City'] = cleaned_df['Plus4']
cleaned_df['States'] = cleaned_df['Station Phone']

cleaned_df.head(10)

/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_22752/2977624054.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['lat'] = cleaned_df['ID']
/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_22752/2977624054.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['lon'] = cleaned_df['Updated At']
/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_22752/2977624054.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

,City,ID,Updated At,Fuel Type Cotion Directions,Plus4,State,Station Phone,lat,lon,Address,Station_Name,Fuel Type,States
0,Montgomery,32.367916,-86.267021,CNG,Montgomery,2951 Chestnut St,AL,32.367916,-86.267021,2951 Chestnut St,Spire - Montgomery Operations Center,CNG,AL
1,Atlanta,33.745843,-84.398837,CNG,Atlanta,340 Whitehall St,GA,33.745843,-84.398837,340 Whitehall St,PS Energy - Atlanta,CNG,GA
2,Atlanta,33.821911,-84.367461,CNG,Atlanta,2424 Piedmont Rd NE,GA,33.821911,-84.367461,2424 Piedmont Rd NE,Metropolitan Atlanta Rapid Transit Authority,CNG,GA
3,Atlanta,33.760256,-84.543822,CNG,Atlanta,270 Marvin Miller Dr,GA,33.760256,-84.543822,270 Marvin Miller Dr,United Parcel Service,CNG,GA
4,Houston,29.779918,-95.435385,CNG,Houston,7721A Washington St,TX,29.779918,-95.435385,7721A Washington St,Clean Energy - Texas Department of Transportation,CNG,TX
5,Fort Smith,35.362213,-94.375338,CNG,Fort Smith,2100 S Waldron Rd,AR,35.362213,-94.375338,2100 S Waldron Rd,Arkansas Oklahoma Gas Corp,CNG,AR
6,East Boston,42.374706,-71.026549,CNG,East Boston,1000 Cottage St Ext,MA,42.374706,-71.026549,1000 Cottage St Ext,Clean Energy - Logan International Airport,CNG,MA
7,Everett,42.393167,-71.064352,CNG,Everett,16 Rover St,MA,42.393167,-71.064352,16 Rover St,Clean Energy - Everett - National Grid,CNG,MA
8,Brooklyn,40.718037,-73.932309,CNG,Brooklyn,287 Maspeth Ave,NY,40.718037,-73.932309,287 Maspeth Ave,Clean Energy - Greenpoint - National Grid,CNG,NY
9,Brooklyn,40.645540,-73.918344,CNG,Brooklyn,8424 Ditmas Ave,NY,40.645540,-73.918344,8424 Ditmas Ave,Canarsie - National Grid,CNG,NY


In [63]:
columns_to_drop = ['ID', 'Updated At','City', 'Plus4', 'Fuel Type Cotion Directions','State','Station Phone']
cleaned_df.drop(columns=columns_to_drop, inplace=True)
final_fuel_df = pd.DataFrame(cleaned_df)
final_fuel_df.head()

/var/folders/ly/6s9xls1s5k92nhwz4573k3vm0000gp/T/ipykernel_22752/1226897146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df.drop(columns=columns_to_drop, inplace=True)


,lat,lon,Address,Station_Name,Fuel Type,States
0,32.367916,-86.267021,2951 Chestnut St,Spire - Montgomery Operations Center,CNG,AL
1,33.745843,-84.398837,340 Whitehall St,PS Energy - Atlanta,CNG,GA
2,33.821911,-84.367461,2424 Piedmont Rd NE,Metropolitan Atlanta Rapid Transit Authority,CNG,GA
3,33.760256,-84.543822,270 Marvin Miller Dr,United Parcel Service,CNG,GA
4,29.779918,-95.435385,7721A Washington St,Clean Energy - Texas Department of Transportation,CNG,TX
